In [186]:
class CareGiver:
    def __init__(self, name, id, trust, approved = False):
        self.m_name = name
        self.m_id = id
        self.m_trust = trust
        self.m_isApproved = approved
        self.m_ratings = {}
        self.m_recipient = []
        self.m_available = False
        self.m_ratingAvg = 0
        self.m_noOfClients = 0
        self.m_trust.m_careGivers.append(self)
        #bisect.insort_left(self.m_trust.m_careGivers, self)
    
    # Getters & Setters... 
    def GetTrusty(self):
        return self.m_trust
    def GetAvailability(self):
        return self.m_available
    def GetName(self):
        return self.m_name
    def GetAvgRating(self):
        return self.m_ratingAvg
    
    #for sorting
    def __lt__(self, other):
        return self.m_ratingAvg < other.m_ratingAvg
    
    def GetARecipients(self):
        # remove later, should not be accessible from outside....
        return self.m_recipient
    def GetApproved(self):
        self.m_isApproved = True
        self.m_available = True
        return
    def SetAvailability(self, available):
        self.m_available = available
        return
        
    def AllocateARecipient(self):
        if(len(self.m_recipient) < 4):
            recipient = self.m_trust.AllocateARecipient(self)
            print(self.m_name, " asked for the allocation of recipient..")
            if recipient != None :
                # care giver may or may not get the recipient now but, since he has made himself, he could be picked by a 
                # recipient...
                self.m_recipient.append(recipient)
                print(self.m_name, " got the allocation of recipient.. : ", recipient.m_name)
                recipient.m_careGiver = self
                self.SetAvailability(False) ## check....
                # above line should ba taken care of in the trust fuction... The mutual connection....
            else: 
                print("Seems no recipient available at the moment..")
        else:
            print("Already have 4 recipients!!")

    def MakeYourselfAvailable(self):
        if len(self.m_recipient) < 4 :
            if self.m_isApproved :
                self.m_available = True
        # When caregiver is fired by the recipient, m_available is set to False, also when caregiver gets assigned new recipient
        # m_available is set to False. Hence, if the caregiver want's more recipient, he has to set m_available to True and also 
        # call the funstion AllocateARecipient()
        return
    
    def QuitARecipient(self, name, rating, reviews = ""):
        # hence the call for the rating should be handdled by the trust. 
        # for now just go with the one way of the rating allotings....
        # we have to remove connections from both sides. Just the way we formed the connection on both sides...
        # remove the entry from the both sides { later do it in the trust functions }
        recipient = next((x for x in self.m_recipient if x.m_name == name), None)
        if(recipient != None):
            
            recipient.m_careGiver = None
            recipient.SetAvailability(True)
            recipient.AllotTheRating(rating, reviews)      ## definr this function in the recipient and caregiver classes.
            self.m_recipient.remove(recipient)             ## Generalize them in the trust class...
            # Do the same thing with FireCareGiver() in Recipient class..... 
        else: 
            print("Not my recipient....")
        return 
        
        
    def AllotTheRating(self, rating, reviews):
        self.m_ratingAvg = (self.m_ratingAvg*self.m_noOfClients + rating)/(self.m_noOfClients + 1)
        self.m_ratings.update( {rating : reviews})
        self.m_noOfClients += 1
        print(self.m_name, " rating alloted")
        return None
        

In [187]:
import bisect 

In [188]:
 class Trust:
        def __init__(self):
            self.m_careGivers = []
            self.m_recipients = []
            
        def ShowAllTheCareGivers(self):
            return self.m_careGivers
        def AllocateARecipient(self, careGiver):
            # Here I might need care_giver info for the ratings..But for now just allocate first available recipient...
            # Here we need to update the content info of the trust as well as the corrosponding caregiver. 
            # See if we only need to update the caregiver info and make it update it's info in the trust container automatically
            # Like a pointer.
            self.m_recipients.sort(key=lambda x: x.m_ratingAvg, reverse=True)
            
            for rec in self.m_recipients :
                if(rec.GetAvailability()):
                    # I should write a function to access this.. 
                    rec.m_available = False
                    return rec
            return None
        
        def AllocateACareGiver(self, recipient):
            self.m_careGivers.sort(key=lambda x: x.m_ratingAvg, reverse=True)
            for cg in self.m_careGivers :
                if(cg.GetAvailability()):
                    if(len(cg.m_recipient) < 4):
                        return cg
            return None
        
        def ShowAllCaregiversData(self):
            for cg in self.m_careGivers :
                print(cg.GetName(), " rating : ", cg.m_ratingAvg)
                for rating in cg.m_ratings :
                    print("\t\t\t", rating, " ", cg.m_ratings[rating])
            return 
        
        def ShowHowIsTakingCareOfRecipients(self):
            for rp in self.m_recipients :
                print(rp.GetName(), " \t\t Care Giver : ", rp.m_careGiver.m_name)
                
            

In [189]:
class Recipient:
    
    def __init__(self, name, id, trust, available = True):
        self.m_name = name
        self.m_id = id;
        self.m_trust = trust
        self.m_available = available
        self.m_careGiver = None
        self.m_ratings = {}
        self.m_ratingAvg = 0
        self.m_noOfClients = 0
        self.m_trust.m_recipients.append(self)
        #bisect.insort_left(self.m_trust.m_recipients, self)
    
    # Getters & Setters...  
    def GetTrusty(self):
        return self.m_trust
    def GetAvailability(self):
        return self.m_available
    def GetName(self):
        return self.m_name
    def SetAvailability(self, available):
        self.m_available = available
        return
    #for sorting
    def __lt__(self, other):
        return self.m_ratingAvg < other.m_ratingAvg
        
    def AllocateACareGiver(self):
        # should be the work of the trust::
        if(self.m_careGiver == None):  # no need to check .. rather if he already has a caregiver then skip 
            careGiver = self.GetTrusty().AllocateACareGiver(self) 
            if(careGiver != None):
                # exchange info and make recipient unavailable. that's it....
                self.m_careGiver = careGiver
                careGiver.m_recipient.append(self)
                self.SetAvailability(False)
                print("Care giver allocated : ", careGiver.m_name)
                #careGiver.m_available = False   #not recipients headache. caregiver can still get upto 4 recipients
            else: 
                    print(" No care givers available at the moment..")
        else:
            print("Already have a caregiver ! : ", self.m_careGiver.m_name) 
        
    def FireTheCareGiver(self, rating, reviews):
        # remove the caregiver info 
        # Make youself available... for next CG 
        if(self.m_careGiver != None):
            self.m_careGiver.m_recipient.remove(self) ## Not sure if this will work sinec, it contains the cycle. 
            self.SetAvailability(True)
            print(self.m_careGiver.m_name, " Just got fired !")
            self.m_careGiver.AllotTheRating(rating, reviews) ## definr this function in the recipient and caregiver classes.
            self.m_careGiver = None
        return
    
    def AllotTheRating(self, rating, reviews):
        self.m_ratingAvg = (self.m_ratingAvg*self.m_noOfClients + rating)/(self.m_noOfClients + 1)
        self.m_noOfClients += 1;
        self.m_ratings.update( {rating : reviews} )

## caregiver should not have the access to recipient info and recipient should not have care givers info.. 
## How to make it happen.. 
## both should not be able to access trust's parameters and functions through m_trust parameter...

In [190]:
theTrust = Trust()
c1 = CareGiver("Tushar", 1, theTrust)
c2 = CareGiver("Prashant", 2, theTrust)
c3 = CareGiver("Adam", 3, theTrust)
c4 = CareGiver("Scott", 4, theTrust)
r1 = Recipient("Al Paccino", 1, theTrust)
r2 = Recipient("Robert", 2, theTrust)
r3 = Recipient("Joe", 3, theTrust)
r4 = Recipient("Perry", 4, theTrust)
r5 = Recipient("Ross", 5, theTrust)
r6 = Recipient("Monica", 6, theTrust)
c1.GetApproved()
c2.GetApproved()
c3.GetApproved()
c1.AllocateARecipient()
c1.AllocateARecipient()
len(c1.m_recipient)
c1.m_recipient[0].m_name

Tushar  asked for the allocation of recipient..
Tushar  got the allocation of recipient.. :  Al Paccino
Tushar  asked for the allocation of recipient..
Tushar  got the allocation of recipient.. :  Robert


'Al Paccino'

In [191]:
c2.AllocateARecipient()

Prashant  asked for the allocation of recipient..
Prashant  got the allocation of recipient.. :  Joe


In [192]:
c3.AllocateARecipient()

Adam  asked for the allocation of recipient..
Adam  got the allocation of recipient.. :  Perry


In [193]:
c2.AllocateARecipient()

Prashant  asked for the allocation of recipient..
Prashant  got the allocation of recipient.. :  Ross


In [194]:
c1.AllocateARecipient()

Tushar  asked for the allocation of recipient..
Tushar  got the allocation of recipient.. :  Monica


In [195]:
c1.QuitARecipient("Monica", 4, "Good lady")

In [197]:
c3.QuitARecipient("Perry", 3, "Good lady")

In [199]:
c2.QuitARecipient("Ross", 1, "Good lady")

In [200]:
c1.QuitARecipient("Al Paccino", 1.5, "Huuaa")

In [202]:
c1.QuitARecipient("Robert", 2, "Good lady")

In [203]:
c1.AllocateARecipient()

Tushar  asked for the allocation of recipient..
Tushar  got the allocation of recipient.. :  Monica


In [204]:
c1.AllocateARecipient()

Tushar  asked for the allocation of recipient..
Tushar  got the allocation of recipient.. :  Perry


In [205]:
c1.AllocateARecipient()

Tushar  asked for the allocation of recipient..
Tushar  got the allocation of recipient.. :  Robert


In [206]:
c1.AllocateARecipient()

Tushar  asked for the allocation of recipient..
Tushar  got the allocation of recipient.. :  Al Paccino


In [208]:
c2.AllocateARecipient()

Prashant  asked for the allocation of recipient..
Prashant  got the allocation of recipient.. :  Ross


In [210]:
c2.AllocateARecipient()

Prashant  asked for the allocation of recipient..
Seems no recipient available at the moment..


In [211]:
c2.m_isApproved

True

In [212]:
len(c1.m_recipient)

4

In [213]:
r6.AllocateACareGiver()

Already have a caregiver ! :  Tushar


In [214]:
c1.MakeYourselfAvailable()

In [215]:
r1.FireTheCareGiver(2, "Not good service")

Tushar  Just got fired !
Tushar  rating alloted


In [216]:
r2.FireTheCareGiver(3, "okkkkeeyyy")

Tushar  Just got fired !
Tushar  rating alloted


In [217]:
r4.FireTheCareGiver(4, "good enough")

Tushar  Just got fired !
Tushar  rating alloted


In [218]:
r5.FireTheCareGiver(5, "Excelante")

Prashant  Just got fired !
Prashant  rating alloted


In [219]:
c1.m_available

False

In [220]:
c1.AllocateARecipient()

Tushar  asked for the allocation of recipient..
Tushar  got the allocation of recipient.. :  Perry


In [221]:
c2.AllocateARecipient()

Prashant  asked for the allocation of recipient..
Prashant  got the allocation of recipient.. :  Robert


In [148]:
c4.MakeYourselfAvailable()

In [151]:
c4.AllocateARecipient()

Scott  asked for the allocation of recipient..
Seems no recipient available at the moment..


#### When a caregiver gets a recipient, normally he would make himself unavailable. But it's his choice. #### If all caregivers ask for one recipient, when new recipient would not find a caregiver. #### Hence caregiver can make himself available upto 4 recipients .....

### testing multiple recipient and multiple firing and reting calculations.. Works fine so far...
r2.FireTheCareGiver(2, "Not good")
r1.FireTheCareGiver(4, "Good")
print(len(c1.m_recipient))
print(c1.m_ratingAvg)

In [224]:
theTrust.ShowHowIsTakingCareOfRecipients()
# why is there None in trust.m_recipient    .. find out

Monica  		 Care Giver :  Tushar
Perry  		 Care Giver :  Tushar
Robert  		 Care Giver :  Prashant


AttributeError: 'NoneType' object has no attribute 'm_name'

In [45]:
print(c2.m_ratingAvg)
r2.FireTheCareGiver(2, "Not good")
print(c2.m_ratingAvg)
r1.FireTheCareGiver(4, "Good")
print(c2.m_ratingAvg)

0
Tushar  Just got fired !
Tushar  rating alloted
0
Tushar  Just got fired !
Tushar  rating alloted
0


In [223]:
theTrust.ShowAllCaregiversData()
## how did prashant get any ratings ?????   Doubt .....
c2.m_noOfClients

Tushar  rating :  3.0
			 2   Not good service
			 3   okkkkeeyyy
			 4   good enough
Prashant  rating :  5.0
			 5   Excelante
Adam  rating :  0
Scott  rating :  0


1

0

## Remember: when caregiver fire the recipient/ recipient fires the caregiver : 
## recipient automatically makes himself available , True
## but caregiver is not available by default. He has to make himself available again to find next recipient.....

##test 3
##fire caregiver of the r1.
##check if the relationsship is updated and also check for the rating update.
r1.FireTheCareGiver(4, "Good service")
print(len(c1.m_recipient))
print(c1.m_noOfClients)

c1.m_trust.m_careGivers[0].m_name  // Cycle 

### we saw that, the update of the caregiver outside reflected in the trust container as well.. Hurreeeyyyy!!!!

###  First part is done, Now lets work on getter and setters..................................................................................................
###  Then let's work on next step. 
####  {May be rating allocation -- Done
####  firing -                                    Done
####  printing all info from the trust    -- Done but, other caregivers infor isn't properly added. 
####  making trust exclusive 
####  making care giver exclusive from the recipients 
####  making the trust singleton...
####  Use ratings to sort and allocate the recipient and caregiver..

### In next draft, make allocation as common function...... And make availability False/True right there in the trust 
#### See if we can make the allocaterecipient/careGiver fucntions more generic....

In [154]:
## for FYI, when recipient fires a caregiver, careGiver is available..... check it..
## also change the design about making yourself availble.... it's oddd...

### Now use rating to maintain the order and see if that works for allocating
1] find a data container that keep the data in order by a data parameter...  
